In [1]:
import hypernetx as hnx
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import sys

In [2]:
virName = "Mers"

In [3]:
#dft = pd.read_pickle("bigTransNo" + virName + ".pkl")
dft = pd.read_pickle("bigTrans" + virName + ".pkl")

In [4]:
# now we use some (though not all) of the from_dataframe() arguments and
# let the function take care of the dataframe manipulation
Hf = hnx.Hypergraph.from_dataframe(dft, # the whole dataframe, b and p columns
                                                    #columns=human_b_cols, # choose specific columns
                                                    zsc='columns', # other option is 'rows'
                                                    absolute=True, # absolute value after z-score is taken
                                                    lower_thresh=2) # applies the > 2 threshold after zscore and absolute value)

# options that I used the defaults for:
# transpose = False: this will transpose the dataframe after z-score and absolute value, essentially creating the dual hypergraph. Instead we're taking the dual after the fact (below).
# name = None (string): If you want to give the resulting hypergraph a "name" attribute. Not necessary.
# key = None (function which evaluates True or False): This is for more complcated thresholding. If you're just doing z-score > some threshold you don't need to worry about this.
# rows = None (list of row names): If you want to use only a subset of the rows. This is done before taking z-score so your z-score will be relative only to those rows chosen.
# upper_thresh = None (number): You can have a maximum value for the the zscore if you want. You can use both upper_thresh and lower_thresh.

In [5]:
Hf.shape

(5229, 49)

In [6]:
HfD = Hf.dual()

In [7]:
HfD.shape

(49, 5229)

In [8]:
graph = HfD

In [9]:
#import ray
#ray.init(num_cpus=3)

In [10]:
#@ray.remote
def betweenness(s, graph = graph):
    sbt = hnx.s_betweenness_centrality(graph, s=s)
    sbt = pd.Series(sbt)
    #sbt.to_pickle("no" + virName + "/" + str(s) + "-betweenness.pkl")
    sbt.to_pickle(virName + "/" + str(s) + "-betweenness.pkl")
    #return(sbt.rename(s))

#@ray.remote
def closeness(s, graph = graph):
    scl = hnx.s_harmonic_closeness_centrality(graph, s=s)
    scl = pd.Series(scl)
    #scl.to_pickle("no" + virName + "/" + str(s) + "-closeness.pkl")
    scl.to_pickle(virName + "/" + str(s) + "-closeness.pkl")
    #return(scl.rename(s))    

In [11]:
#HfDsbt2 = ray.get([betweenness.remote(i) for i in range(1,graph.shape[0])])
#HfDscl2 = ray.get([closeness.remote(i) for i in range(1,graph.shape[0])])

In [12]:
#ray.shutdown()

In [13]:
for i in range(1,graph.shape[0]):
    betweenness(i)